In [58]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000) # Generate dataset

In [59]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size) 

In [60]:
next(iter(data_iter)) # Get a batch of data

[tensor([[-1.5051, -1.1053],
         [-1.5727, -0.7618],
         [-0.8675,  1.1443],
         [-0.6939, -0.4537],
         [-0.4038,  0.4112],
         [ 0.3398,  0.1551],
         [-0.3286,  0.6102],
         [ 0.4090, -0.5419],
         [ 0.8576, -0.2527],
         [-0.1665, -0.2877]]),
 tensor([[ 4.9400],
         [ 3.6539],
         [-1.4433],
         [ 4.3711],
         [ 2.0080],
         [ 4.3606],
         [ 1.4877],
         [ 6.8428],
         [ 6.7873],
         [ 4.8559]])]

In [61]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1)) # Define a neural network with one linear layer

In [72]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0) #方差

tensor([0.])

In [ ]:
loss = nn.MSELoss() #计算均方误差使用的是MSELoss类，也称为平方L2范数。 默认情况下，它返回所有样本损失的平均值。

# reduction='sum' 总损失；
# reduction='mean' 平均损失

In [74]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03) # Define the optimizer

In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()

        trainer.step()
    l = loss(net(features), labels)
    # 访问梯度
    weight_grad = net[0].weight.grad  #访问梯度
    bias_grad = net[0].bias.grad      #访问梯度
    
    print("权重梯度:", weight_grad)
    print("偏置梯度:", bias_grad)
    print(f'epoch {epoch + 1}, loss {l:f}')

权重梯度: tensor([[-0.0073,  0.0031]])
偏置梯度: tensor([0.0051])
epoch 1, loss 0.000097
权重梯度: tensor([[-0.0046,  0.0107]])
偏置梯度: tensor([0.0071])
epoch 2, loss 0.000099
权重梯度: tensor([[0.0017, 0.0018]])
偏置梯度: tensor([-0.0042])
epoch 3, loss 0.000098


In [76]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([ 0.0006, -0.0004])
b的估计误差： tensor([0.0002])


练习1
需要将学习率乘以小批量的大小 batch_size。

练习2
PyTorch 中常见的损失函数 (Loss Functions)
nn.MSELoss(): 均方误差损失，常用于回归任务。
nn.CrossEntropyLoss(): 交叉熵损失，常用于分类任务。
nn.L1Loss(): 平均绝对误差 (MAE)。
nn.NLLLoss(): 负对数似然损失。
nn.SmoothL1Loss(): 平滑 L1 损失，对异常值更鲁棒。类似 Huber Loss

PyTorch 中常见的初始化方法 (Initialization Methods)
torch.nn.init.normal_(): 从正态分布中采样初始化。
torch.nn.init.uniform_(): 从均匀分布中采样初始化。
torch.nn.init.constant_(): 用常数初始化。
torch.nn.init.xavier_normal_() / xavier_uniform_(): Xavier 初始化，有助于保持梯度在各层间稳定流动。
torch.nn.init.kaiming_normal_() / kaiming_uniform_(): He 初始化，特别适合 ReLU 激活函数。


1. 损失函数
函数	核心用途	关键特点
MSELoss	无异常值的回归	梯度平滑，对异常值敏感
L1Loss	有异常值的回归	鲁棒，梯度不连续
CrossEntropy	分类任务（标配）	结合 Softmax，直接用 logits
Huber Loss	有异常值的回归	结合 MSE 和 MAE 的优点
2. 初始化方法
方法	适配激活函数	核心场景
normal_/uniform_	线性层 / 简单网络	基础初始化（比如线性回归）
Xavier	Sigmoid/Tanh	传统神经网络
He	ReLU/LeakyReLU	深度网络（CNN/Transformer）
constant_	偏置项	固定初始值（比如偏置 = 0）
3. Huber Loss 核心
对小误差用 MSE（平滑），大误差用 MAE（鲁棒）；
PyTorch 中SmoothL1Loss就是 δ=1 的 Huber Loss，直接用更方便；
适合回归任务有异常值的场景。



In [83]:
import torch
import torch.nn as nn

# 方法1：自定义Huber Loss（易理解）
def huber_loss(y_hat, y, delta=1.0):
    error = y_hat - y
    abs_error = torch.abs(error)
    # 小误差用MSE，大误差用MAE
    loss = torch.where(abs_error <= delta, 
                       0.5 * error**2, 
                       delta * (abs_error - 0.5 * delta))
    return loss.mean()  # 返回批次平均损失

# 方法2：用PyTorch内置的SmoothL1Loss（等价于delta=1的Huber Loss）
huber_loss_pytorch = nn.SmoothL1Loss()

# 测试示例
y_true = torch.tensor([2.0, 3.0, 5.0])  # 真实值（含一个异常值5.0）
y_pred = torch.tensor([1.8, 3.2, 10.0]) # 预测值（第三个是明显异常）

# 自定义Huber Loss
loss_custom = huber_loss(y_pred, y_true)
# 内置SmoothL1Loss
loss_pytorch = huber_loss_pytorch(y_pred, y_true)

print(f"自定义Huber Loss: {loss_custom.item():.4f}")  
print(f"内置SmoothL1Loss: {loss_pytorch.item():.4f}") 

自定义Huber Loss: 1.5133
内置SmoothL1Loss: 1.5133


练习3
 PyTorch 中，当你调用 loss.backward() 后，模型参数（权重和偏置）的梯度就会被计算并存储在参数的 .grad 属性中。
 如上[75]的#代码